In [1]:
from selenium import webdriver
import base64, pickle, time
from IPython.display import Image
from IPython.display import display
from PIL import Image as pil

### 1. Get Youtube Images

##### (1) Get Youtube Driver & Save Element Screen

In [2]:
def getDriver(youtube_url):
    driver =  webdriver.Chrome()
    driver.get(youtube_url)
    return driver

def saveScreen(driver, filename):
    element = driver.find_element_by_css_selector('#movie_player')
    location = element.location
    size = element.size
    driver.save_screenshot('screenshot.png')
    im = pil.open('screenshot.png')
    left = location['x']
    top = location['y'] * 2
    right = left + size['width'] * 2
    bottom = top + size['height'] * 2

    # left, top, right, bottom
    im = im.crop((left, top, right, bottom)) 
    im.save(filename) 

##### (2) Check Element

In [3]:
def checkElement(driver, selector):
    result = True
    try:
        driver.find_element_by_css_selector(selector)
    except:
        result = False
    return result

##### (3) Skip Ad

In [4]:
skipAd_selector = ".videoAdUiSkipButton.videoAdUiAction.videoAdUiFixedPaddingSkipButton"
closeAd_selector = ".adDisplay.extra-padding .close-button"

def skipAd(driver):
    if checkElement(driver, skipAd_selector):
        driver.find_element_by_css_selector(skipAd_selector).click()
    
def closeAd(driver):
    if checkElement(driver, closeAd_selector):
        driver.find_element_by_css_selector(closeAd_selector).click()

### 2. Google Vision API

In [23]:
# single thread
def vision(filenames):
    url = 'https://cloud.google.com/vision/'

    # open site
    driver =  webdriver.Chrome()
    driver.get(url)

    # enter iframe
    iframe = driver.find_element_by_css_selector("#vision_demo_section iframe")
    driver.switch_to_frame(iframe)
    
    result = []
    
    for filename in filenames:
    
        # file upload
        driver.find_element_by_id("input").send_keys(filename)

        # check done analytics
        delay = 0
        for _ in range(10):
            if driver.find_element_by_css_selector("#results").text != '':
                break
            time.sleep(1)
            delay += 1
        # print("Analytics Delay Time : {} sec".format(delay))
        
        if delay >= 10:
            continue
        
        # check face
        isFace = True
        try:
            driver.find_element_by_css_selector('[data-type="faceAnnotations"]')
        except:
            isFace = False

        face_data_list = []

        if isFace:
            face_list = driver.find_elements_by_css_selector('.face.style-scope.vs-faces')
            for idx, face in enumerate(face_list):
                faceData = {
                    "name":"face " + str(idx+1),
                    "confidence": face.find_element_by_css_selector('.confidence .conf-score.style-scope.vs-faces').text
                }          
                expressions = face.find_elements_by_css_selector('.type-row.style-scope.vs-faces')
                expressionData = {}
                for expression in expressions:
                    name = expression.find_element_by_css_selector('#label').text
                    state = expression.find_element_by_css_selector('#text').text
                    if "Likely" in state or "Possible" in state:
                        state = 1
                    else:
                        state = 0
                    expressionData[name] = state
                faceData["expressions"] = expressionData
                face_data_list.append(faceData)  
                
        result.append(face_data_list)    
        
        # close image
        driver.find_element_by_css_selector("#exit").click()
    driver.close()
    return result

In [6]:
def saveImages(youtube_url, start_term, image_count, save_term):
    
    # open driver
    driver = getDriver(youtube_url)

    # skip ad
    time.sleep(6)
    skipAd(driver)

    # save images
    time.sleep(start_term)
    for idx in range(1, image_count+1):
        saveScreen(driver, './data/images/screenshot' + str(idx) + '.png')
        time.sleep(save_term)
        print('saved : screenshot' + str(idx) + '.png')
    # close driver
    driver.close()

In [13]:
%%time
youtube_url = "" # 이미지 생성 youtube URL 
start_term = 25 # 20초 후 부터 이미지 생성
image_count = 50 # 10개의 이미지 생성
save_term = 0.1 # 저장 텀
saveImages(youtube_url, start_term, image_count, save_term)

saved : screenshot1.png
saved : screenshot2.png
saved : screenshot3.png
saved : screenshot4.png
saved : screenshot5.png
saved : screenshot6.png
saved : screenshot7.png
saved : screenshot8.png
saved : screenshot9.png
saved : screenshot10.png
saved : screenshot11.png
saved : screenshot12.png
saved : screenshot13.png
saved : screenshot14.png
saved : screenshot15.png
saved : screenshot16.png
saved : screenshot17.png
saved : screenshot18.png
saved : screenshot19.png
saved : screenshot20.png
saved : screenshot21.png
saved : screenshot22.png
saved : screenshot23.png
saved : screenshot24.png
saved : screenshot25.png
saved : screenshot26.png
saved : screenshot27.png
saved : screenshot28.png
saved : screenshot29.png
saved : screenshot30.png
saved : screenshot31.png
saved : screenshot32.png
saved : screenshot33.png
saved : screenshot34.png
saved : screenshot35.png
saved : screenshot36.png
saved : screenshot37.png
saved : screenshot38.png
saved : screenshot39.png
saved : screenshot40.png
saved : s

In [14]:
# make image list
filenames = []
for idx in range(1, image_count + 1):
    filename = '/Users/rada/Documents/code/git/crawling/data/images/screenshot' + str(idx) + '.png'
    filenames.append(filename)

In [16]:
# make thumnail
size = (512, 512)
thumnails = []
for filename in filenames:
    im = pil.open(filename)
    im.thumbnail(size)
    savename = filename.split("/")
    savename[len(savename)-1] = savename[len(savename)-1].replace(".png","_thumbnail.png")
    savename = "/".join(savename)
    im.save(savename, "png")
    thumnails.append(savename)

In [42]:
%%time
result = vision(thumnails)

Analytics Delay Time : 3 sec
Analytics Delay Time : 10 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 2 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 10 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 2 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 10 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics Delay Time : 3 sec
Analytics D

In [47]:
person_count = 0
scene_count = 0
expressions = {'Joy':0, 'Sorrow':0, 'Anger':0, 'Surprise':0, 'Exposed':0, 'Blurred':0, 'Headwear':0}
expression_list = ['Joy', 'Sorrow', 'Anger', 'Surprise', 'Exposed', 'Blurred', 'Headwear']

for data in result:
    scene_count += 1
    person_count += len(data)
    for face in data:
        for expression in expression_list:
            expressions[expression] = expressions[expression] + face['expressions'][expression]

In [48]:
scene_count, person_count, expressions

(44,
 23,
 {'Anger': 2,
  'Blurred': 0,
  'Exposed': 0,
  'Headwear': 8,
  'Joy': 16,
  'Sorrow': 0,
  'Surprise': 1})